In [1]:
from surprise import  KNNBasic,KNNWithMeans,KNNWithZScore,KNNBaseline,SVD,SVDpp,NMF,BaselineOnly,CoClustering

from surprise import Dataset
from surprise.accuracy import rmse, mae, fcp
from surprise import Reader

import pandas as pd
import pickle
from collections import defaultdict

from function import get_top_n, surprise_precision_recall_at_k
import optuna

In [ ]:
random_state=42

Surprise используется в различных областях для построения и оценки рекомендательных систем. Вот некоторые из них:

* Электронная коммерция: Рекомендации товаров и услуг на основе предыдущих покупок и просмотров пользователей.
* Платформы потокового вещания: Рекомендации фильмов, сериалов и музыки на основе предпочтений пользователей и их взаимодействий.
* Социальные сети: Персонализированные рекомендации друзей, групп и контента.
* Образование: Рекомендации курсов и учебных материалов на основе прошлых успехов и интересов студентов.
* Новости и контент: Рекомендации новостных статей и других информационных материалов на основе предпочтений пользователей.

Surprise помогает создавать точные и персонализированные рекомендации, улучшая опыт пользователей в различных сферах.

In [3]:
# # # Производим десериализацию и извлекаем из файла формата pkl
with open('data/train_time.pkl', 'rb') as pkl_file:
    train = pickle.load(pkl_file)

with open('data/test_time.pkl', 'rb') as pkl_file:
    test = pickle.load(pkl_file)

In [4]:
# train = train.sample(frac=0.2, random_state=random_state)

In [5]:
event_type = {
            'view': 1,
            'addtocart':2,
            'transaction': 10,
            }

test['event'] = test['event'].map(event_type)
train['event'] = train['event'].map(event_type)

In [6]:
train = train.groupby(['visitorid','itemid'],as_index=False)['event'].sum()
test = test.groupby(['visitorid','itemid'],as_index=False)['event'].sum()

In [7]:
train = train[["itemid", "visitorid", "event"]].rename(columns={"itemid": "iid","visitorid": "uid",'event':'r_ui'})
test = test[["itemid", "visitorid", "event"]].rename(columns={"itemid": "iid","visitorid": "uid",'event':'r_ui'})

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3313 entries, 0 to 3312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   iid     3313 non-null   int64
 1   uid     3313 non-null   int64
 2   r_ui    3313 non-null   int64
dtypes: int64(3)
memory usage: 77.8 KB


In [9]:
train

,iid,uid,r_ui
0,10034,172,4
1,27248,172,1
2,464731,172,3
3,19278,419,14
4,243949,1032,15
...,...,...,...
7143,7943,1405831,14
7144,182785,1405861,17
7145,19269,1406087,27
7146,436004,1406981,13


In [10]:
test.r_ui.max()

224

Классы в surprise принимают данные через свой отдельный класс Dataset, в который мы отправляем данные и класс Reader с указанным диапазоном допустимых оценок:

In [11]:
# Создание Reader и Dataset
reader = Reader(rating_scale=(1, 300))
train_data = Dataset.load_from_df(train[['uid', 'iid', 'r_ui']], reader)
test_data = Dataset.load_from_df(test[['uid', 'iid', 'r_ui']], reader)

# Создание полного тренировочного набора
trainset = train_data.build_full_trainset()

# Преобразование тестового набора в формат списков кортежей
raw_testset = [(uid, iid, r_ui) for (uid, iid, r_ui) in zip(test['uid'], test['iid'], test['r_ui'])]

Виды моделей в библиотеке surprise
* KNNBasic: Простой алгоритм ближайших соседей (K-Nearest Neighbors).
* KNNWithMeans: Алгоритм KNN, который учитывает средние рейтинги пользователей или предметов.
* KNNWithZScore: KNN алгоритм с нормализацией Z-оценок.
* KNNBaseline: KNN алгоритм с базовыми линиями (baseline).
* SVD: Алгоритм сингулярного разложения матрицы (Singular Value Decomposition).
* SVDpp: Улучшенная версия SVD, учитывающая дополнительные предпочтения пользователей.
* NMF: Негативное матричное факторизация (Non-negative Matrix Factorization).
* BaselineOnly: Базовый алгоритм для прогнозирования рейтингов.
* CoClustering: Алгоритм совместной кластеризации.

**Fraction of Concordant Pairs (FCP)** – это метрика для оценки качества предсказательных моделей, в частности, рекомендательных систем. FCP измеряет долю пар прогнозов, которые имеют правильный порядок относительно друг друга.  
Конкордантная пара – это пара, в которой предсказания модели соответствуют истинным рейтингам. Например, если у нас есть два товара и предсказание модели совпадает с фактическими рейтингами (более высокий рейтинг предсказан для товара с более высоким истинным рейтингом), это конкордантная пара.  
Эта метрика помогает понять, насколько хорошо модель предсказывает правильный порядок элементов, что важно для ранжирования в рекомендательных системах.  
Чем выше FCP, тем лучше модель в правильном упорядочивании элементов. Высокий FCP указывает на то, что модель делает больше правильных предсказаний порядка оценок, что очень важно для ранжирования в рекомендательных системах.

In [12]:
models = [
          KNNBasic(random_state=random_state),
          KNNWithMeans(random_state=random_state),
          KNNWithZScore(random_state=random_state),
          KNNBaseline(random_state=random_state),
          SVD(random_state=random_state),
          SVDpp(random_state=random_state),
          NMF(random_state=random_state),
          BaselineOnly(),
          CoClustering(random_state=random_state)
        ]
result_rmse = []
result_mae = []
result_fcp = []


for model in models:
    # print(model)
    model.fit(trainset)
    test_pred = model.test(raw_testset)
    result_rmse.append(rmse(test_pred))
    result_mae.append(mae(test_pred))
    result_fcp.append(fcp(test_pred))


models_name = []
for i in range(len(models)):
    models_name.append(str(models[i]))


df_rez = pd.DataFrame(data=[models_name, result_rmse, result_mae, result_fcp]).T
df_rez.columns = ['model', 'rmse', 'mae', 'fcp']
df_rez

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 9.9085
MAE:  7.2557
FCP:  0.5204
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 10.0234
MAE:  7.2258
FCP:  0.5566
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 10.0509
MAE:  7.2353
FCP:  0.5494
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 9.7901
MAE:  6.9625
FCP:  0.5368
RMSE: 9.6910
MAE:  6.9442
FCP:  0.5395
RMSE: 9.6449
MAE:  7.0645
FCP:  0.5167
RMSE: 9.2815
MAE:  6.8307
FCP:  0.5539
Estimating biases using als...
RMSE: 8.8851
MAE:  6.6401
FCP:  0.5305
RMSE: 10.5696
MAE:  7.1535
FCP:  0.5537


,model,rmse,mae,fcp
0,<surprise.prediction_algorithms.knns.KNNBasic ...,9.90846,7.25574,0.520447
1,<surprise.prediction_algorithms.knns.KNNWithMe...,10.023437,7.225778,0.556552
2,<surprise.prediction_algorithms.knns.KNNWithZS...,10.050933,7.235334,0.549436
3,<surprise.prediction_algorithms.knns.KNNBaseli...,9.790065,6.962484,0.536846
4,<surprise.prediction_algorithms.matrix_factori...,9.690959,6.944181,0.53946
5,<surprise.prediction_algorithms.matrix_factori...,9.644938,7.064539,0.516702
6,<surprise.prediction_algorithms.matrix_factori...,9.281474,6.830706,0.553923
7,<surprise.prediction_algorithms.baseline_only....,8.885082,6.640058,0.530522
8,<surprise.prediction_algorithms.co_clustering....,10.569616,7.153454,0.553732


In [13]:
# Получим топ рекомендаций для visitorid == 172
uid = 172
get_top_n(test_pred,10)[uid]

[10034, 465522]

Попробую использовать BaselineOnly модель и подобрать для нее наилучшие гиперпараметры.

Параметры BaselineOnly:

* bsl_options: Опции для настройки базовой линии.  
* method: Метод, используемый для расчета базовой линии. Возможные значения:  
    * 'als': Алгоритм чередующихся наименьших квадратов.  
    * 'sgd': Стохастический градиентный спуск.  
* n_epochs: Количество эпох обучения (только для метода sgd). По умолчанию 20.  
* reg_u: Регуляризация для смещений пользователей. По умолчанию 15 для метода als и 0.02 для метода sgd.  
* reg_i: Регуляризация для смещений элементов. По умолчанию 10 для метода als и 0.02 для метода sgd.  
* learning_rate: Скорость обучения (только для метода sgd). По умолчанию 0.005.  

In [14]:
def opt_Ext(trial):
    # задаем пространство поиска гиперпараметров
    method = trial.suggest_categorical('method',['als','sgd'])
    n_epochs = trial.suggest_categorical('n_epochs',[3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,25,30,])
    reg_u = trial.suggest_categorical('reg_u',[0.001,0.005,0.006,0.007,0.008,0.009,0.01,0.02,0.03, 0.05,0.1,0.5,1,3,5,7,10,11,12,13,14,15,17,20,25,30,])
    reg_i = trial.suggest_categorical('reg_i',[0.001,0.005,0.006,0.007,0.008,0.009,0.01,0.02,0.03, 0.05,0.1,0.5,1,3,5,7,10,11,12,13,14,15,17,20,25,30,])

    # создаем модель
    model = BaselineOnly(
                        verbose=0,
                        bsl_options={'method':method,
                                     'n_epochs':n_epochs,
                                     'reg_u':reg_u,
                                     'reg_i':reg_i
                                     }
                        )

    model.fit(trainset)
    test_pred = model.test(raw_testset)
    score = fcp(test_pred)
    return score

In [15]:
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо минимизировать метрику direction="minimize"
optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna.logging.set_verbosity(optuna.logging.ERROR)
stud = optuna.create_study(direction="maximize")


# ищем лучшую комбинацию гиперпараметров
stud.optimize(opt_Ext, n_trials=500)

FCP:  0.5314
FCP:  0.5289
FCP:  0.5339
FCP:  0.5281
FCP:  0.5338
FCP:  0.5375
FCP:  0.5361
FCP:  0.5365
FCP:  0.5369
FCP:  0.5361
FCP:  0.5330
FCP:  0.5309
FCP:  0.5370
FCP:  0.5301
FCP:  0.5275
FCP:  0.5375
FCP:  0.5375
FCP:  0.5372
FCP:  0.5291
FCP:  0.5334
FCP:  0.5318
FCP:  0.5375
FCP:  0.5375
FCP:  0.5325
FCP:  0.5375
FCP:  0.5365
FCP:  0.5435
FCP:  0.5284
FCP:  0.5435
FCP:  0.5435
FCP:  0.5435
FCP:  0.5435
FCP:  0.5435
FCP:  0.5435
FCP:  0.5399
FCP:  0.5398
FCP:  0.5443
FCP:  0.5326
FCP:  0.5420
FCP:  0.5499
FCP:  0.5499
FCP:  0.5499
FCP:  0.5499
FCP:  0.5499
FCP:  0.5499
FCP:  0.5499
FCP:  0.5499
FCP:  0.5473
FCP:  0.5499
FCP:  0.5399
FCP:  0.5375
FCP:  0.5499
FCP:  0.5499
FCP:  0.5399
FCP:  0.5296
FCP:  0.5499
FCP:  0.5475
FCP:  0.5308
FCP:  0.5290
FCP:  0.5386
FCP:  0.5498
FCP:  0.5499
FCP:  0.5329
FCP:  0.5432
FCP:  0.5314
FCP:  0.5479
FCP:  0.5329
FCP:  0.5334
FCP:  0.5473
FCP:  0.5407
FCP:  0.5320
FCP:  0.5499
FCP:  0.5326
FCP:  0.5396
FCP:  0.5419
FCP:  0.5327
FCP:  0.5315

In [16]:
print(stud.best_params)

{'method': 'als', 'n_epochs': 17, 'reg_u': 0.001, 'reg_i': 0.1}


In [17]:
best_model_surprise = BaselineOnly(
                                verbose=0,
                                bsl_options={'method':stud.best_params['method'],
                                            'n_epochs':stud.best_params['n_epochs'],
                                            'reg_u':stud.best_params['reg_u'],
                                            'reg_i':stud.best_params['reg_i'],
                                            })

best_model_surprise.fit(trainset)


# # Производим сериализацию и записываем результат в файл формата pkl
with open(r'models\best_model_surprise.pkl', 'wb') as output:
    pickle.dump(best_model_surprise, output)

In [18]:
fig = optuna.visualization.plot_contour(stud,)
fig

In [19]:
predictions = best_model_surprise.test(raw_testset)

In [20]:
precision_at_k, recall_at_k = surprise_precision_recall_at_k(predictions, k=3, threshold=4)

# Precision can then be averaged over all users
print(sum(prec for prec in precision_at_k.values()) / len(precision_at_k))
# print(sum(rec for rec in recall_at_k.values()) / len(recall_at_k))

0.5626272912423623
